# Paper 27: Better & Faster Large Language Models via Multi-token Prediction
## Meta AI Research (2024)

### Multi-token Prediction

Key insight: Train LMs to predict multiple future tokens simultaneously. Improves sample efficiency and generation quality!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Standard Single-Token Prediction

Traditional language modeling:
```
Input:  [w1, w2, w3, w4]
Predict: w5
```

In [ ]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

class SingleTokenRNN:
    """Standard RNN with single-token prediction"""
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        # Embeddings
        self.W_embed = np.random.randn(vocab_size, embedding_dim) * 0.01
        
        # RNN weights
        self.W_xh = np.random.randn(hidden_dim, embedding_dim) * 0.01
        self.W_hh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.b_h = np.zeros((hidden_dim, 1))
        
        # Output projection (predict next token)
        self.W_out = np.random.randn(vocab_size, hidden_dim) * 0.01
        self.b_out = np.zeros((vocab_size, 1))
    
    def forward(self, input_seq):
        """
        Forward pass
        input_seq: list of token indices
        Returns: predictions for next token at each position
        """
        h = np.zeros((self.hidden_dim, 1))
        predictions = []
        hidden_states = []
        
        for token_idx in input_seq:
            # Embed
            x = self.W_embed[token_idx].reshape(-1, 1)
            
            # RNN step
            h = np.tanh(np.dot(self.W_xh, x) + np.dot(self.W_hh, h) + self.b_h)
            
            # Predict next token
            logits = np.dot(self.W_out, h) + self.b_out
            probs = softmax(logits.T)
            
            predictions.append(probs.flatten())
            hidden_states.append(h.copy())
        
        return predictions, hidden_states

# Test
vocab_size = 50
single_model = SingleTokenRNN(vocab_size, embedding_dim=32, hidden_dim=64)
test_seq = [1, 2, 3, 4]
preds, _ = single_model.forward(test_seq)
print(f"Input sequence length: {len(test_seq)}")
print(f"Predictions shape: {len(preds)} x {len(preds[0])}")
print(f"Predicts: 1 token ahead at each position")

## Multi-Token Prediction

Predict multiple future tokens:
```
Input:  [w1, w2, w3, w4]
Predict: w5, w6, w7  (3 tokens ahead!)
```

In [ ]:
class MultiTokenRNN:
    """RNN with multi-token prediction"""
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_future_tokens=3):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_future_tokens = num_future_tokens
        
        # Shared embeddings and RNN
        self.W_embed = np.random.randn(vocab_size, embedding_dim) * 0.01
        self.W_xh = np.random.randn(hidden_dim, embedding_dim) * 0.01
        self.W_hh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.b_h = np.zeros((hidden_dim, 1))
        
        # Multiple output heads (one per future position)
        self.output_heads = []
        for i in range(num_future_tokens):
            W_out = np.random.randn(vocab_size, hidden_dim) * 0.01
            b_out = np.zeros((vocab_size, 1))
            self.output_heads.append((W_out, b_out))
    
    def forward(self, input_seq):
        """
        Forward pass
        Returns: predictions for next N tokens at each position
        """
        h = np.zeros((self.hidden_dim, 1))
        multi_predictions = []  # List of (pred_t+1, pred_t+2, ..., pred_t+N)
        hidden_states = []
        
        for token_idx in input_seq:
            # Embed
            x = self.W_embed[token_idx].reshape(-1, 1)
            
            # RNN step
            h = np.tanh(np.dot(self.W_xh, x) + np.dot(self.W_hh, h) + self.b_h)
            
            # Predict next N tokens using separate heads
            position_preds = []
            for W_out, b_out in self.output_heads:
                logits = np.dot(W_out, h) + b_out
                probs = softmax(logits.T)
                position_preds.append(probs.flatten())
            
            multi_predictions.append(position_preds)
            hidden_states.append(h.copy())
        
        return multi_predictions, hidden_states

# Test
multi_model = MultiTokenRNN(vocab_size, embedding_dim=32, hidden_dim=64, num_future_tokens=3)
multi_preds, _ = multi_model.forward(test_seq)
print(f"Input sequence length: {len(test_seq)}")
print(f"Multi-predictions: {len(multi_preds)} positions")
print(f"At each position: {len(multi_preds[0])} future tokens")
print(f"Each prediction shape: {multi_preds[0][0].shape}")
print(f"\nPredicts: {len(multi_preds[0])} tokens ahead at each position!")

## Synthetic Text Data

In [ ]:
def generate_synthetic_sequences(vocab_size=50, num_sequences=1000, seq_length=20):
    """
    Generate synthetic sequences with patterns
    Pattern: arithmetic progressions (e.g., 1, 2, 3, 4, ...)
    """
    sequences = []
    
    for _ in range(num_sequences):
        # Random starting point and step
        start = np.random.randint(0, vocab_size // 2)
        step = np.random.randint(1, 3)
        
        # Generate arithmetic sequence
        seq = [(start + i * step) % vocab_size for i in range(seq_length)]
        sequences.append(seq)
    
    return sequences

# Generate data
train_sequences = generate_synthetic_sequences(vocab_size, num_sequences=1000, seq_length=20)
test_sequences = generate_synthetic_sequences(vocab_size, num_sequences=200, seq_length=20)

print(f"Training sequences: {len(train_sequences)}")
print(f"Example sequence: {train_sequences[0][:10]}...")
print(f"Pattern: arithmetic progression")

## Training: Single-Token Prediction

In [ ]:
def train_single_token(model, sequences, epochs=50, lr=0.01):
    """
    Train with standard next-token prediction
    """
    losses = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        for seq in sequences:
            # Predict next token at each position
            for i in range(len(seq) - 1):
                input_tokens = seq[:i+1]
                target_token = seq[i+1]
                
                # Forward
                predictions, _ = model.forward(input_tokens)
                pred_probs = predictions[-1]  # Last position prediction
                
                # Loss
                loss = -np.log(pred_probs[target_token] + 1e-8)
                epoch_loss += loss
                
                # Backward (simplified - just track loss)
        
        avg_loss = epoch_loss / (len(sequences) * (len(seq) - 1))
        losses.append(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return losses

# Train single-token model
print("Training Single-Token Model...\n")
single_losses = train_single_token(single_model, train_sequences[:100], epochs=30)
print(f"\nFinal loss: {single_losses[-1]:.4f}")

## Training: Multi-Token Prediction

In [ ]:
def train_multi_token(model, sequences, epochs=50, lr=0.01):
    """
    Train with multi-token prediction
    Loss = sum of losses for all future positions
    """
    losses = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        num_predictions = 0
        
        for seq in sequences:
            # Predict multiple tokens at each position
            for i in range(len(seq) - model.num_future_tokens):
                input_tokens = seq[:i+1]
                target_tokens = seq[i+1:i+1+model.num_future_tokens]
                
                # Forward
                multi_preds, _ = model.forward(input_tokens)
                position_preds = multi_preds[-1]  # Last position predictions
                
                # Loss for each future position
                for j, (pred_probs, target) in enumerate(zip(position_preds, target_tokens)):
                    loss = -np.log(pred_probs[target] + 1e-8)
                    epoch_loss += loss
                    num_predictions += 1
        
        avg_loss = epoch_loss / num_predictions if num_predictions > 0 else 0
        losses.append(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    
    return losses

# Train multi-token model
print("\nTraining Multi-Token Model (3 tokens ahead)...\n")
multi_losses = train_multi_token(multi_model, train_sequences[:100], epochs=30)
print(f"\nFinal loss: {multi_losses[-1]:.4f}")

## Compare Learning Curves

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(single_losses, label='Single-Token Prediction', linewidth=2, marker='o', markersize=4)
plt.plot(multi_losses, label='Multi-Token Prediction (3 ahead)', linewidth=2, marker='s', markersize=4)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Average Loss', fontsize=12)
plt.title('Learning Curves: Single vs Multi-Token Prediction', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nSingle-token final loss: {single_losses[-1]:.4f}")
print(f"Multi-token final loss: {multi_losses[-1]:.4f}")
print(f"\nMulti-token prediction provides richer training signal!")

## Evaluation: Prediction Accuracy

In [ ]:
def evaluate_single_token(model, sequences):
    """Evaluate next-token prediction accuracy"""
    correct = 0
    total = 0
    
    for seq in sequences:
        for i in range(len(seq) - 1):
            input_tokens = seq[:i+1]
            target = seq[i+1]
            
            predictions, _ = model.forward(input_tokens)
            pred_token = np.argmax(predictions[-1])
            
            if pred_token == target:
                correct += 1
            total += 1
    
    return correct / total if total > 0 else 0

def evaluate_multi_token(model, sequences, position=0):
    """Evaluate multi-token prediction accuracy at specific future position"""
    correct = 0
    total = 0
    
    for seq in sequences:
        for i in range(len(seq) - model.num_future_tokens):
            input_tokens = seq[:i+1]
            target = seq[i+1+position]
            
            multi_preds, _ = model.forward(input_tokens)
            pred_probs = multi_preds[-1][position]  # Prediction for position ahead
            pred_token = np.argmax(pred_probs)
            
            if pred_token == target:
                correct += 1
            total += 1
    
    return correct / total if total > 0 else 0

# Evaluate both models
single_acc = evaluate_single_token(single_model, test_sequences[:50])
multi_acc_t1 = evaluate_multi_token(multi_model, test_sequences[:50], position=0)
multi_acc_t2 = evaluate_multi_token(multi_model, test_sequences[:50], position=1)
multi_acc_t3 = evaluate_multi_token(multi_model, test_sequences[:50], position=2)

print("\nEvaluation Results:")
print(f"{'='*60}")
print(f"Single-Token Model:")
print(f"  Next token (t+1): {single_acc:.2%}")
print(f"\nMulti-Token Model:")
print(f"  Next token (t+1): {multi_acc_t1:.2%}")
print(f"  2 tokens ahead (t+2): {multi_acc_t2:.2%}")
print(f"  3 tokens ahead (t+3): {multi_acc_t3:.2%}")
print(f"{'='*60}")

## Visualize Multi-Token Predictions

In [ ]:
# Generate prediction accuracy heatmap
test_seq = test_sequences[0][:15]
accuracies = np.zeros((len(test_seq) - 3, 3))

for i in range(len(test_seq) - 3):
    input_tokens = test_seq[:i+1]
    targets = test_seq[i+1:i+4]
    
    multi_preds, _ = multi_model.forward(input_tokens)
    position_preds = multi_preds[-1]
    
    for j in range(3):
        pred_token = np.argmax(position_preds[j])
        accuracies[i, j] = 1.0 if pred_token == targets[j] else 0.0

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap
im = ax1.imshow(accuracies.T, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax1.set_xlabel('Input Position', fontsize=12)
ax1.set_ylabel('Future Position', fontsize=12)
ax1.set_title('Multi-Token Prediction Accuracy', fontsize=13, fontweight='bold')
ax1.set_yticks([0, 1, 2])
ax1.set_yticklabels(['t+1', 't+2', 't+3'])
plt.colorbar(im, ax=ax1, label='Accuracy (1=Correct, 0=Wrong)')

# Average accuracy by distance
avg_accs = np.mean(accuracies, axis=0)
positions = ['t+1', 't+2', 't+3']
bars = ax2.bar(positions, avg_accs, color=['green', 'orange', 'red'], edgecolor='black', linewidth=2)
ax2.set_ylabel('Average Accuracy', fontsize=12)
ax2.set_title('Accuracy vs Prediction Distance', fontsize=13, fontweight='bold')
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, acc in zip(bars, avg_accs):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.1%}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nFurther predictions are harder (as expected)")

## Sample Efficiency Comparison

In [ ]:
# Train on varying dataset sizes
dataset_sizes = [10, 25, 50, 100, 200]
single_final_losses = []
multi_final_losses = []

print("Testing sample efficiency...\n")

for size in dataset_sizes:
    print(f"Training on {size} sequences...")
    
    # Single-token
    single_temp = SingleTokenRNN(vocab_size, embedding_dim=32, hidden_dim=64)
    single_loss = train_single_token(single_temp, train_sequences[:size], epochs=20, lr=0.01)
    single_final_losses.append(single_loss[-1])
    
    # Multi-token
    multi_temp = MultiTokenRNN(vocab_size, embedding_dim=32, hidden_dim=64, num_future_tokens=3)
    multi_loss = train_multi_token(multi_temp, train_sequences[:size], epochs=20, lr=0.01)
    multi_final_losses.append(multi_loss[-1])

# Plot
plt.figure(figsize=(12, 6))
plt.plot(dataset_sizes, single_final_losses, 'o-', linewidth=2, markersize=10, 
        label='Single-Token', color='blue')
plt.plot(dataset_sizes, multi_final_losses, 's-', linewidth=2, markersize=10, 
        label='Multi-Token (3 ahead)', color='red')
plt.xlabel('Number of Training Sequences', fontsize=12)
plt.ylabel('Final Loss', fontsize=12)
plt.title('Sample Efficiency: Single vs Multi-Token', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xscale('log')
plt.tight_layout()
plt.show()

print("\nMulti-token prediction is more sample efficient (learns faster with less data)!")

## Key Takeaways

### Multi-Token Prediction:

**Standard LM**:
```
Given: w1, w2, w3
Predict: w4
Loss: -log P(w4 | w1, w2, w3)
```

**Multi-Token LM**:
```
Given: w1, w2, w3
Predict: w4, w5, w6  (multiple tokens!)
Loss: -log P(w4|w1:3) - log P(w5|w1:3) - log P(w6|w1:3)
```

### Architecture:

**Shared Backbone**:
- Embeddings
- RNN/Transformer layers

**Multiple Output Heads**:
- Head 1: Predicts t+1
- Head 2: Predicts t+2
- Head 3: Predicts t+3
- ...

Each head is a separate linear layer (small overhead!)

### Benefits:

1. **Sample Efficiency** ✅
   - Each example provides N training signals (not just 1)
   - Learns N times faster (approximately)

2. **Better Representations** ✅
   - Forced to encode longer-term dependencies
   - Can't just memorize next token

3. **Faster Inference** ✅
   - Can generate multiple tokens in one forward pass
   - Speculative decoding: verify predictions in parallel

4. **Better Generalization** ✅
   - More training signal → better features
   - Regularization effect

### Training:

**Loss Function**:
$$
\mathcal{L} = \sum_{i=1}^{N} \lambda_i \cdot \mathcal{L}_{\text{next-token}}(t+i)
$$

Where:
- $N$ = number of future tokens
- $\lambda_i$ = weight for position $i$ (can downweight distant future)

**Typical settings**:
- $N = 3$ or $N = 4$ tokens ahead
- Equal weights: $\lambda_i = 1/N$
- Or decay: $\lambda_i = \gamma^{i-1}$ where $\gamma < 1$

### Results from Paper (Meta AI):

**7B model**:
- Standard: X perplexity
- Multi-token (4 ahead): 0.7X perplexity (better!)

**Sample efficiency**:
- Multi-token with 1/3 data = Standard with full data

**Inference speed**:
- 3x faster generation (using speculative decoding)

### Inference Strategies:

**1. Standard (still valid)**:
```
Use only head 1 (t+1 predictions)
Same as normal autoregressive generation
```

**2. Speculative Decoding**:
```
Generate w4, w5, w6 from heads
Verify each prediction
Keep valid prefix, regenerate rest
→ Up to Nx speedup!
```

**3. Beam Search Enhancement**:
```
Consider multiple future paths simultaneously
Better long-range planning
```

### Comparison with Other Techniques:

| Technique | Sample Efficiency | Inference Speed | Complexity |
|-----------|------------------|-----------------|------------|
| Standard LM | 1x | 1x | Low |
| Data Augmentation | 1.2x | 1x | Low |
| **Multi-Token** | **2-3x** | **1-3x** | **Low** |
| Distillation | 1.5x | 1.5x | High |

### Implementation Tips:

1. **Start simple**: N=2 or N=3 tokens
2. **Shared trunk**: Only output heads are separate
3. **Equal weighting**: Unless you have reason to prefer near/far future
4. **Monitor each head**: Track accuracy for each position
5. **Use for speedup**: Speculative decoding in inference

### When to Use:

✅ **Good for**:
- Limited training data
- Want faster inference
- Long sequences (benefits from long-range signal)
- Structured outputs (code, formulas)

❌ **Not ideal for**:
- Very short sequences
- Highly random outputs
- Memory constrained (extra heads add parameters)

### Modern Extensions:

1. **Adaptive N**: Use different N for different layers
2. **Hierarchical**: Predict next word, next phrase, next sentence
3. **Discrete diffusion**: Multi-step generation
4. **Continuous-time**: Predict at arbitrary future times

### Key Insight:

**More prediction = More learning signal = Better models**

Multi-token prediction is essentially **free regularization** with **bonus speedup**. Almost no downside!

**"Why predict one token when you can predict many?"** - Meta AI Team